<a href="https://colab.research.google.com/github/GrantMcCurdy/BikeShare/blob/main/Bike_Share_UserType_Classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Cleaning

In [160]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [161]:
df = pd.read_excel('/content/2020_10_Cleaned.xlsx')

In [162]:
df = df[['rideable_type', 'ride_length', 'started_at_time', 'ride_length', 'day_of_week', 'member_casual']].copy()

In [163]:
ride_time = [] 

for i in range(len(df['started_at_time'])):
  converted_time = df['started_at_time'][i].hour + df['started_at_time'][i].minute / 60.0
  ride_time.append(converted_time)


ride_period = []

for i in range(len(ride_time)):
  if ride_time[i] >= 0 and ride_time[i]  < 3:
    ride_period.append('late night')
  elif ride_time[i]  >= 3 and ride_time[i]  < 6:
    ride_period.append('early morning')
  elif ride_time[i]  >= 6 and ride_time[i]  < 9:
    ride_period.append('late morning')
  elif ride_time[i]  >= 9 and ride_time[i]  < 12:
    ride_period.append('early day')
  elif ride_time[i]  >= 12 and ride_time[i]  < 15:
    ride_period.append('late dat')
  elif ride_time[i] >= 15 and ride_time[i]  < 18:
    ride_period.append('early evening')
  elif ride_time[i]  >= 18 and ride_time[i]  < 21:
    ride_period.append('late evening')
  elif ride_time[i]  >= 21 and ride_time[i]  < 24:
    ride_period.append('early night')
  else:
    ride_period.append('null')

df['ride_period'] = ride_period

In [ ]:
ride_day = []

for i in range(len(df['day_of_week'])):
  if df['day_of_week'][i] == 1:
    ride_day.append('Mon')
  elif df['day_of_week'][i] == 2:
    ride_day.append('Tues')
  elif df['day_of_week'][i] == 3:
    ride_day.append('Wed')
  elif df['day_of_week'][i] == 4:
    ride_day.append('Thurs')
  elif df['day_of_week'][i] == 5:
    ride_day.append('Fri')
  elif df['day_of_week'][i] == 6:
    ride_day.append('Sat')
  elif df['day_of_week'][i] == 7:
    ride_day.append('Sun')
  else:
    ride_day.append('Null')
    
df['ride_day'] = ride_day

In [ ]:
df = df[['rideable_type', 'ride_length', 'ride_period', 'ride_day', 'member_casual' ]]

In [ ]:
df = df.loc[:,~df.columns.duplicated()]

##Data Preprocessing

In [ ]:
#Data Preprocessing
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [ ]:
#Label encode rideable_type column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 0] = le.fit_transform(X[:, 0])

In [ ]:
#Label encode member_casual
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
#One Hot Encode ride_period
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
#One Hot Encode ride_day
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [-1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
#Splitting dataset into Training and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

##ANN Classifier

In [ ]:
#Initializing ANN
ann = tf.keras.models.Sequential()

In [ ]:
#Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units = 12, activation = 'relu'))

In [ ]:
#Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units = 12, activation = 'relu'))

In [ ]:
#Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units = 12, activation = 'relu'))

In [ ]:
#Adding the output layer
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [ ]:
#Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#training the ANN on the Training Set
ann.fit(X_train, y_train, batch_size = 32, epochs = 30)

In [ ]:
#Predicting Test set results
y_pred_ann = ann.predict(X_test)
y_pred_ann = (y_pred > 0.5)

In [ ]:
#Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_ann)
print(cm)
accuracy_score(y_test, y_pred_ann)

##Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfw_classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy')
rfw_classifier.fit(X_train, y_train)

In [ ]:
y_pred_rfw = rfw_classifier.predict(X_test)

In [ ]:
#Confusion Matrix and Accuracy
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_rfw)
print(cm)
accuracy_score(y_test, y_pred_rfw)

##Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(criterion = 'entropy')
dt_classifier.fit(X_train, y_train)

In [ ]:
y_pred_dt = dt_classifier.predict(X_test)

In [ ]:
#Confusion Matrix and Accuracy
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_dt)
print(cm)
accuracy_score(y_test, y_pred_dt)

Out of the models tested, ANN is the most accurate with a score of almost .70 (a decent model).